## Note: This is the code I used to calculate the free energies with MBAR. 
I then implemented the code in the re-eds pipeline but the .npy files
which contain the final free energies for each system were calculated with this script

In [339]:
import reeds
import pygromos

import numpy as np
import glob as glob
from scipy import stats
from scipy import constants as const

from scipy.special import logsumexp


from mpmath import * # This is for floating point arithmetic ! 
mp.dps = 15

import copy

import pymbar
from pymbar import MBAR
from pymbar.timeseries import subsample_correlated_data

In [340]:
from reeds.function_libs.file_management import file_management as fM
#from reeds.function_libs.analysis import analysis as ana

from pygromos.files.imd import Imd

# 0) Define the data to work on

In [355]:
PROJECT = 'NIK'
FF = 'complex_gaff'

project_path = f'/fileserver/pine/pine2/cchampion/REEDS/{PROJECT}/{FF}'

print (project_path)

if 'complex' in project_path:
    NAME_END = f'{PROJECT}_{FF}'
else:
    NAME_END = f'{PROJECT}_water_{FF}'
    
print (NAME_END)

/fileserver/pine/pine2/cchampion/REEDS/NIK/complex_gaff
NIK_complex_gaff


In [358]:
prod_paths = [f'f_prod_seed{i}' for i in range(1, 6)]
print(prod_paths)

imd = Imd(f'{project_path}/{prod_paths[0]}/input/repex_prod.imd')

s_values = np.array(imd.REPLICA_EDS.RES, dtype=float)
eoffs = np.array(imd.REPLICA_EDS.EIR, dtype=float).T

num_states = len(eoffs[0])
print (num_states)

['f_prod_seed1', 'f_prod_seed2', 'f_prod_seed3', 'f_prod_seed4', 'f_prod_seed5']
6


In [359]:
def get_prefix(path_data):
    filename = glob.glob(f'{path_data}/*energies*.dat')[0]
    return '_'.join(filename.split('/')[-1].split('_')[0:-1])
    

In [361]:
ene_trajs_all_seeds = []

for prod in prod_paths:
    prefix = get_prefix(f'{project_path}/{prod}/analysis/data/')
    ene_trajs = fM.parse_csv_energy_trajectories(in_folder=f'{project_path}/{prod}/analysis/data/', ene_trajs_prefix=prefix)
        
    # For NIK remove equilibration and extra little end 
    ene_trajs = [traj[500*5:2500*5:5] for traj in ene_trajs]
    
    ene_trajs_all_seeds.append(ene_trajs)

# 1) Do the M-BAR analysis

In [362]:
def reformat_trajs4mbar_with_decorrelation(ene_trajs, s_values, eoffs, num_states, l = 1, temp = 298):
    """
    Reformats a gromos energy trajectory to fit the expected input of pymbar (u_kln)
    
    This function also re-evaluates the reference potential at other values of s for the calculation.
    
    # We would need to add to that the initial trajectories to have the A -> R, B -> R, etc.
    
    # TO-DO: Rework the code so it automatically subsamples 
             each trajectory to remove correlated datapoints(do that based on sampled V_R)
    
    Input
    ------
        
        l = number of replicas to use in addition!
    e_prod_seed1/analysis/data
    """
    
    temp = 298
    kt = (temp * const.k * const.Avogadro) / 1000
    beta =  1 / kt
    
    end_states = [f'e{i+1}' for i in range(num_states)]
    
    # a little bit ugly we do work twice here
    n = [len(subsample_correlated_data(traj['eR'])) for traj in ene_trajs[:l]]
    idx_ns = np.append([0], np.cumsum(n))
    
    k_tot = num_states + l # we will always have the additional (l) V_R states
    
    u_kn = np.zeros([k_tot, np.sum(n)]) # energies evaluated at all states k for all n samples
    N_k = np.zeros(k_tot) # number of samples from states k
    
    for i, traj in enumerate(ene_trajs):
        if i == l:
            break
        
        # print (f'working on replica # {i+1}')
        beg = idx_ns[i]
        end = idx_ns[i+1]
        #rint (f'{beg}-{end}')
        
        # Reformat the data
        vr = np.array(traj['eR'])
        idx_subsample = subsample_correlated_data(vr)
        
        vr = vr[idx_subsample]
        vis = np.array(traj[end_states])[idx_subsample]

        # Add the potential energies of the end states
        for k, vk in enumerate(vis.T): 
            u_kn[k, beg:end] = vk
        
        # Add the potential energies of the reference states at different s-values
        for k in range(num_states, num_states+l):
            idx_params = (k-num_states)
            if idx_params == i:
                u_kn[k, beg:end] = vr
            else: #recalc ref potential at other s values = s_i
                s = s_values[idx_params]
                _eoffs = eoffs[idx_params]                
                expterm =  - (beta*s) * np.subtract(vis,  _eoffs).T
                u_kn[k, beg:end] = -1/(beta*s) * logsumexp(expterm, axis=0)
            
        N_k[i+num_states] = len(vr)
    
    # Convert to reduced potential energies
    u_kn *= beta
    
    return u_kn, N_k 

In [363]:
def reformat_trajs4mbar_without_decorrelation(ene_trajs, s_values, eoffs, num_states, l = 1, temp = 298):
    """
    Reformats a gromos energy trajectory to fit the expected input of pymbar (u_kln)
    
    This function also re-evaluates the reference potential at other values of s for the calculation.
    
    # We would need to add to that the initial trajectories to have the A -> R, B -> R, etc.
    
    # TO-DO: Rework the code so it automatically subsamples 
             each trajectory to remove correlated datapoints(do that based on sampled V_R)
    
    Input
    ------
        
        l = number of replicas to use in addition!
    
    """
    
    temp = 298
    kt = (temp * const.k * const.Avogadro) / 1000
    beta =  1 / kt
    
    end_states = [f'e{i+1}' for i in range(num_states)]
    
    # a little bit ugly we do work twice here
    n = [len(ene_trajs[0]['eR'])] * l
    
    idx_ns = np.append([0], np.cumsum(n))
    
    print (n)
    print (idx_ns)
    
    k_tot = num_states + l # we will always have the additional (l) V_R states
    
    u_kn = np.zeros([k_tot, np.sum(n)]) # energies evaluated at all states k for all n samples
    N_k = np.zeros(k_tot) # number of samples from states k
    
    for i, traj in enumerate(ene_trajs):
        if i == l:
            break
        
        # print (f'working on replica # {i+1}')
        beg = idx_ns[i]
        end = idx_ns[i+1]
        #rint (f'{beg}-{end}')
        
        # Reformat the data
        vr = np.array(traj['eR'])
        idx_subsample = subsample_correlated_data(vr)
        
        vr = vr[idx_subsample]
        vis = np.array(traj[end_states])[idx_subsample]

        # Add the potential energies of the end states
        for k, vk in enumerate(vis.T): 
            u_kn[k, beg:end] = vk
        
        # Add the potential energies of the reference states at different s-values
        for k in range(num_states, num_states+l):
            idx_params = (k-num_states)
            if idx_params == i:
                u_kn[k, beg:end] = vr
            else: #recalc ref potential at other s values = s_i
                s = s_values[idx_params]
                _eoffs = eoffs[idx_params]                
                expterm =  - (beta*s) * np.subtract(vis,  _eoffs).T
                u_kn[k, beg:end] = -1/(beta*s) * logsumexp(expterm, axis=0)
            
        N_k[i+num_states] = len(vr)
    
    # Convert to reduced potential energies
    u_kn *= beta
    
    return u_kn, N_k 

# Analysis of the trajectory using all of the data 

In [364]:
temp = 298
kt = (temp * const.k * const.Avogadro) / 1000
beta =  1 / kt

percents = np.arange(100, 101, 5)

num_points = len(percents)
num_seeds = len(ene_trajs_all_seeds)

mbar_convergence = np.zeros((num_points, num_seeds, num_states)) # num_slices, num_seeds, sum_deltaGs
zwanzig_convergence = np.zeros((num_points, num_seeds, num_states)) # num_slices, num_seeds, sum_deltaGs

size_ene = len(ene_trajs[0])

for j, ene_trajs in enumerate(ene_trajs_all_seeds):
    for i, percent in enumerate(percents):
        imax = int(size_ene * percent/100)
        tmp =  [ t[0:imax] for t in copy.deepcopy(ene_trajs)]
                
        u_kn, N_k = reformat_trajs4mbar_with_decorrelation(tmp, s_values, eoffs, num_states = num_states, l=32)
        # u_kn, N_k = reformat_trajs4mbar_without_decorrelation(tmp, s_values, eoffs, num_states = num_states, l=32)
        mbar = MBAR(u_kn, N_k)
        
        try:
            results = mbar.compute_free_energy_differences()
            mbar_convergence[i,j] = results['Delta_f'][num_states][0:num_states] * kt
        except:
            print ('error')
        
        # Return free energies w.r.t. R (s=1) (1->R, 2->5, ..., N->R)
         
        #u_kn, N_k = reformat_trajs4mbar_with_decorrelation(tmp, s_values, eoffs, num_states = num_states, l=1)
        #mbar = MBAR(u_kn, N_k)
        #results = mbar.compute_free_energy_differences()
        
        #zwanzig_convergence[i,j] = results['Delta_f'][num_states][0:num_states]
        
        #zwanzig_convergence[i,j] = results['Delta_f'][num_states][0:num_states] * kt 
        
        pass

# Save the data to a numpy array 

In [349]:
#### note index 0 here is because we used data from a 3D array (so we recover data using 100% of sim) 
# it also assumes percent is set accordingly in cell above: percents = np.arange(100, 101, 5)

In [365]:
final_results = np.array([np.average(mbar_convergence[0], axis=0), np.std(mbar_convergence[0], axis=0)])
final_results2 = np.array([np.average(zwanzig_convergence[0], axis=0), np.std(zwanzig_convergence[0], axis=0)])

# overwrite results for PIM complex_gaff state 4:

#final_results[0][3] = (198.79344633 + 197.62367259 + 202.83888035 + 203.99023936) / 4
#final_results[1][3] = np.std([198.79344633, 197.62367259, 202.83888035, 203.99023936])

In [366]:
final_results[0]

array([   2.28287239,   33.17065415, -240.84953026,  131.52512623,
         44.51909273, -212.64904664])

In [352]:
final_results2[0]

array([0., 0., 0., 0., 0., 0.])

In [368]:
path_arr = f'/fileserver/pine/pine2/cchampion/REEDS/{PROJECT}/results/{NAME_END}.npy'

np.save(path_arr, final_results)

In [367]:
project_path

'/fileserver/pine/pine2/cchampion/REEDS/NIK/complex_gaff'